<font size="6">ELECTRE Tri-B step-by-step</font>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/cghiaus/ELECTRE_Tri/main?labpath=docs%2Fhow_to_guides%2Fstep_by_step.ipynb)

ELECTRE Tri-B is a sorting method for multiple-criteria decision-making ([MCDM](https://en.m.wikipedia.org/wiki/Multiple-criteria_decision_analysis)) in which alternatives are assigned to categories. The categories are completely ordered and defined by base (or reference) profiles ([Figueira et al. 2005](https://doi.org/10.1007/0-387-23081-5_4)). 

Note that ELECTRE Tri-B uses boundary profiles to define categories: each category is defined by a lower and upper limiting profile. ELECTRE Tri-C uses central (characteristic) reference actions to define categories: each category is represented by a single central reference action that typifies that category ([Bouyssou, Marchant 2015]( https://doi.org/10.1016/j.ejor.2014.09.057)).

In [1]:
"""
Append `src/` directory to `path`
"""
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(os.path.dirname(notebook_dir))

src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

In [2]:
import pandas as pd

import electre_tri as et

# Introduction

ELECTRE is a family of **outranking** methods. The relation **"a outranks b"** (denoted as $a > b$, $a\, S\, b$, or $a \succ b$) means that there are sufficient arguments to support the claim that **alternative $a$ is at least as good as alternative $b$**, while there are no strong arguments against this claim.

## Assumptions

1. The set of categories to which the alternatives must be assigned to is completely ordered (from the worst to the best, from the lowest to the highest priority, from the  riskiest to the least risky, etc.).
2. Each category is defined to receive alternatives (or actions), which are processed in the same way.
3. Each category is defined by two reference alternatives, which represent its lower and upper bounds, called boundary reference alternatives, boundary actions or base profiles. The lower base profile of a better category is also the upper base profile of the worse consecutive category.

## Definitions
  
### Input data

- **Criteria** $c = \{c_k\}$ are the different factors or attributes used to evaluate the alternatives in the decision making process. The criteria can be maximised (e.g. efficiency, savings) or minimised (e.g. cost, environmental impact).

- **Alternatives** are the options or items being evaluated and sorted into categories. These represent the decision objects that need to be classified according to multiple criteria. The performance of each alternative $a_i$ across all criteria $c_k$ form the **performance matrix** $A = \{a_{i,k}\}$ with $i=0,\ldots,n-1$ and $k=0,\ldots,m-1$, where $i$ is the index of alternatives and $k$ is the index of criteria. In the case of a criterion $k$ with an increasing preference direction, the higher the evaluation of the alternative on this criterion, $a_{i,k}$, the better the alternative performs on this criterion. Conversely, for a criterion $k$ with a decreasing performance direction, the lower the evaluation of the alternative on this criterion $a_{i,k}$, the lower the performance of this alternative on this criterion. In order to unify the calculations and not to have to differentiate between the two cases described above, the performance values in the criteria with a decreasing preference direction will be multiplied by "-1". Thus, these criteria will also get an increasing performance direction. 

- **Base profiles** (boundary reference actions, boundary actions, or reference profiles) define the boundaries between the predefined ordered **categories**. The base profiles form a matrix: $B = \{b_{j,k}\}$ with $j=0,\ldots,l-1$ and $k=0,\ldots,m-1$, where $j$ is the index of base profile and $k$ is the index of criteria. The base profile values for criteria with a decreasing preference direction will be multiplied by "-1".

- **Thresholds** to account for uncertainnty and imprecision of alternatives and base profiles $T = \{t_{j,k}\}$, where $k$ is the index of criteria and $j=\{q, p, v\}$ are the vectors:

    - *indifference* $q = \{q_k \}$ : The largest difference in performance on a criterion that a decision maker considers negligible.
    - *preference* $p = \{p_k\}$ : The smallest difference in performance on a criterion that is considered significant for a clear preference.
    - *veto* $v = \{v_k\}$ : A difference in performance so large that it can negate any outranking relationship indicated by other criteria; the relation $q_k \leq p_k \leq v_k$ holds for each $k.$


- **Criteria weights** $w = \{w_k\}$  represent the relative importance of each criterion in the decision-making process.

- **Credibility threshold** $\lambda$ is a cutting level value that determines whether the outranking relation between an alternative and a profile is considered valid or not. It is usually set within the range of 0.5 to 1 (typically 0.75).

### Concepts specific to ELECTRE problem solving
- **Partial concordances** $c(a, b)$ between profiles $a$ and $b$ for each criterion $c$ is the truth value (between 0 and 1) of the concordance (i.e. agreement) with the statement *a outranks b for criterion c* where "outranks" means "is at least as good as". The comparison is done between alternative $a_{i,k}$ and base $b_{j,k}$ for each criterion $c_k$. The result, $c_{(k,j),i}(a_{i,k},b_{j,k})$, has a double index  (criterion $k$, base profile $j$) and columns corresponding to alternatives $i.$

- **Global concordance** $C(a,b)$ between profiles $a$ and $b$ is the truth value (berween 0 and 1) of the statement *a outranks b globally, i.e. for all criteria* where "outranks" means "is at least as good as". The global concordances are calculated by using the partial concordances and the weights. They are the normalized weighted sum of partial concordances $c_{(k,j),i}$ for each criterion.

- **Partial discordances** $d(a, b)$ between profiles $a$ and $b$ for each criterion $c$ is the truth value (between 0 and 1) of the discordance (i.e. disagreement) with the statement: *a outranks b for criterion c* where "outranks" means "is at least as good as". The comparison is done between alternative $a_{i,k}$ and base profile $b_{j,k}$ for each criterion $c_k.$

- **Credibility index** $\sigma(a, b)$ represents how credible the assertion *a outranks b* is. It can be seen as a fuzzy relation of outranking. The credibility index $\sigma(a,b)$ corresponds to the global concordance index $C(a,b)$ weakened by partial discordances $d(a, b).$

- **Outranking relationships** $a_i$ `I` $b_j$, $a_i$ `>` $b_j$, $a_i$  `<` $b_j$ and $a_i$ `R` $b_j$ (for indiferrent, preferred, not preferred, and incomparable) are relations between alternatives $a$ and $b$ derived from the credibility indexes $\sigma(a, b)$ and $\sigma(b, a).$

- **Optimistic classification** is a procedure in which the level of base profiles is increased till the lowest base profile which is "preferred" to alternative (b > a) is found. The alternative is assigned to the category having this base as upper bound. If no base is "preferred", the alternative is assigned to the category above the highest base profile.

- **Pessimistic classification** is a procedure in which the level of base profiles is decreased till a base profile which is "not preferred" to alternative (b < a) is found. The alternative is assigned to the category having this base as lower bound. If no base is "not preferred", the alternative is assigned to the category below the lowest base profile.

### Results

- **Ranking** means indicating for each alternative the category to which it belongs.

- **Sorting** means indicating for each category the alternatives that belong to it.

# Problem statement for ELECTRE Tri MCDM

ELECTRE Tri is a multiple-criteria decision-making (MCDM) method designed to sort a set of alternatives into predefined ordered categories based on their performance across multiple criteria, considering both supporting and opposing arguments for each assignment decision. 

The problem it addresses can be stated as:

**Given:**
- A set of criteria $c = \{c_k\}$ with associated weights $w = \{w_k\}$.
- A matrix $A = \{a_{i,k}\}$ of performances of alternatives $a_i$ for each criterion $k$.
- A matrix $B = \{b_{j,k}\}$ of base profiles (or category boundaries) $b_j$ for each criterion $k$ that form a set of $l$ ordered categories $C$, where $l - 1$ is the number of base profiles.
- Preference $p = \{p_k\}$, indifference $q = \{q_k\}$, and veto $v = \{v_k\}$ threshold vectors for each criterion.
- A credibility threshold $\lambda$ for outranking.


**Do:**

Assign each alternative $a_i ∈ A$ to one of the predefined categories $C_j ∈ C$ based on its performance across all criteria $c_k$ by using optimistic and pessimistic classification, while considering:

1. The outranking relations between alternatives and base profiles.
2. The credibility of these outranking relations.
3. The concordance and discordance between alternatives and base profiles.
4. The possibility of indifference and/or incomparability between alternatives and base profiles.

# Features of ELECTRE Tri-B

The method provides a robust sorting procedure that can handle:
- Multiple conflicting criteria.
- Imprecise or uncertain information.
- Partial compensation between criteria.
- Preference, indifference, and veto thresholds.

In [3]:

data_file = '../../data/base_profile.csv'
A, B, T, w = et.read_electre_tri_data(data_file)

In [4]:
print("Performance of alternatives")
A

Performance of alternatives


,c1,c2
a1,8.5,18.0
a2,14.0,16.0
a3,5.0,27.0


In [5]:
print("Base profiles")
B

Base profiles


,c1,c2
b1,10.0,15.0
b2,15.0,20.0


In [6]:
print("Thresholds")
T

Thresholds


,c1,c2
q,1.0,2.0
p,2.0,4.0
v,4.0,8.0


In [7]:
print("Weights")
w.to_frame().T.rename(index={0: 'w'})

Weights


,c1,c2
w,0.7,0.3


In [8]:
credibility_threshold = 0.7

# Problem solving

Given:
- criteria to maximize<sup>*</sup>; 
- alternatives, __A__, and increasing base profiles, __B__;
- thresholds (preference, indifference, and veto), __T__;
- weights, __w__;
- credibility threshold, λ.

Calculate:
- partial concordance and discordance indexes;
- global concordance indexes;
- credibility indexes;
- outranking relations;

and perform classification using:
- optimistic (ascending) procedure;
- pessimistic (descending) procedure.

<sup>*</sup>_Note_: The sorting problem is solved for criteria to maximize. To deal with criteria that need to be minimized, multiply by $-1$ the data for these criteria, i.e. the columns of alternatives __A__ and base profiles __B__ (but not of theresholds __T__ and weights __w__) corresponding to these criteria.

## Partial concordance

Partial concordance between profiles $a$ and $b$ for each criterion $c$ is the truth value (between 0 and 1) of the concordance (i.e. agreement) with the statement:

$$a \text{ outranks } b \text{ for criterion } c$$

where "outranks" means "is at least as good as".

In [9]:
c_ab, c_ba = et.partial_concordance(A, B, T)

### Partial concordance between alternatives and base profiles

For a given criterion $k$:
- If the alternative is higher than the base minus the indifference, then the concordance is 1, i.e. we agree that "$a$ outranks $b$" is true.

- If the alternative is lower than the base minus the preference, then the concordance is 0, i.e. we don't agree that "$a$ outranks $b$".

- Otherwise, the concordance is between 0 and 1, i.e. we partially agree that "$a$ outranks $b$".

In [10]:
print("\nPartial concordance \nc_ab = \n")
c_ab


Partial concordance 
c_ab = 



a1   a2  a3
criteria base              
c1       b1    0.5  1.0   0
         b2    0.0  1.0   0
c2       b1    1.0  1.0   1
         b2    1.0  0.0   1

### Partial concordance between base profiles and alternatives

For a given criterion $k$:
- If the base  is higher than the alternative minus the indifference, then the concordance is 1, i.e. we agree that "$b$ outranks $a$" is true.

- If the base is lower than the alternative minus the preference, then the concordance is 0, i.e. we don't agree that "$b$ outranks $a$".

- Otherwise, the concordance is between 0 and 1, i.e. we partially agree that "$b$ outranks $a$".

## Discordance

Partial discordance between profiles `a` and `b` for each criterion `c` is the truth value (between 0 and 1) of the discordance (i.e. disagreement) with the statement:

$$a \text{ outranks } b \text{ for criterion } c$$

where "outranks" means "is at least as good as".

In [11]:
d_ab, d_ba = et.discordance(A, B, T)
d_ab

a1  a2  a3
criteria base            
c1       b1     0   0   1
         b2     1   0   1
c2       b1     0   0   0
         b2     0   0   0

### Discordance between alternatives and base profiles

For a given criterion $k$,

- If the alternative is higher than the base minus the preference, then the discordance is 0, i.e. we do not disagree that "$a$ outranks $b$" is true.

- If the alternative is lower than the base minus the veto, then the concordance is 1, i.e. we disagree that "$a$ outranks $b$".

- Otherwise, the discordance is between 0 and 1, i.e. we partially disagree that "$a$ outranks $b$".

In [12]:
print("\nPartial discordance between alternatives and base profiles \nd_ab = \n")
d_ab


Partial discordance between alternatives and base profiles 
d_ab = 



a1  a2  a3
criteria base            
c1       b1     0   0   1
         b2     1   0   1
c2       b1     0   0   0
         b2     0   0   0

### Discordance between base profiles and alternatives

For a given criterion $k$,

- If the base is higher than the alternative minus the preference, then the discordance is 0, i.e. we do not disagree that "$b$ outranks $a$" is true.

- If the base is lower than the alternative minus the veto, then the concordance is 1, i.e. we disagree that "$b$ outranks $a$".

- Otherwise, the discordance is between 0 and 1, i.e. we partially disagree that "$b$ outranks $a$".

## Global concordance

Global concordance between profiles $a$ and $b$ is the truth value (between 0 and 1) of the statement:
$$a \text{ outranks } b \text{ globally, i.e. for all criteria}$$
where "outranks" means "is at least as good as".

In [13]:
C_ab = et.global_concordance(c_ab, w)
C_ab

,a1,a2,a3
base,,,
b1,0.65,1.0,0.3
b2,0.3,0.7,0.3


In [14]:
C_ba = et.global_concordance(c_ba, w)
C_ba

,a1,a2,a3
base,,,
b1,0.85,0.3,0.7
b2,1.0,1.0,0.7


## Credibility index of outranking relation

Credibility index represents how credible the assertion "$a$ outranks $b$" is. It can be seen as a fuzzy relation of outranking. The credibility index $\sigma(a,b)$ corresponds to the global concordance index $C(a,b)$ weakened by discordances $d(a, b)$:

- When no criterion shows strong opposition (discordance) to the outranking relation, the credibility index is equal to the global concordance.

- When a discordant criterion opposes a veto to the assertion ”$a$ outranks $b$" (i.e. discordance is 1), then credibility index $\sigma(a,b)$ becomes null (the assertion ”$a$ outranks $b$" is not credible at all).

- When one or more criteria strongly oppose the outranking relation (i.e., their discordance exceeds the global concordance), the credibility index is reduced by multiplying the global concordance by a factor derived from the discordances that exceed the global concordance. The formula for this correction involves a product of terms, each representing the effect of a discordant criterion.

In [15]:
sigma_ab = et.credibility_index(C_ab, d_ab)
sigma_ab

,a1,a2,a3
base,,,
b1,0.65,1.0,0.0
b2,0.0,0.7,0.0


In [16]:
sigma_ba = et.credibility_index(C_ba, d_ba)
sigma_ba

,a1,a2,a3
base,,,
b1,0.85,0.0,0.0
b2,1.0,1.0,0.583333


## Outranking relationships

The fuzzy outranking relations, expressed by the credibility indexes $\sigma$, is transformed into an outranking relationship  by using a cutting level $\lambda$.

There are four types of relationships that can be established between each $a_i$ and each $b_j$:

- $a_i$  `I`  $b_j$ : $a_i$  is indifferent to  $b_j$ if $\sigma(a_i, b_j) \geq \lambda$ and $\sigma(b_j, a_i) \geq \lambda$ 
- $a_i$  `>`  $b_j$ : $a_i$  is preferred to  $b_j$ if $\sigma(a_i, b_j) \geq \lambda$ and $\sigma(b_j, a_i) < \lambda$ 
- $a_i$  `<`  $b_j$ : $a_i$  is not preferred to  $b_j$ if $\sigma(a_i, b_j) < \lambda$ and $\sigma(b_j, a_i) \geq \lambda$ 
- $a_i$  `R`  $b_j$ : $a_i$  incomparable to $b_j$ if $\sigma(a_i, b_j) < \lambda$ and $\sigma(b_j, a_i) < \lambda$

|                    | $\sigma(a_i, b_j) \geq \lambda$ |$\sigma(a_i, b_j) < \lambda$|
| ---------------------------------|-------------------|----------------------------|
| $\sigma(b_j, a_i) \geq \lambda$  | $a_i$  `I`  $b_j$ | $a_i$  `<`  $b_j$ |
| $\sigma(b_j, a_i) < \lambda$     | $a_i$  `>`  $b_j$ | $a_i$  `R`  $b_j$ |


In [17]:
outranking = et.outrank(sigma_ab, sigma_ba, credibility_threshold=0.7)
outranking

,a1,a2,a3
base,,,
b1,≻,≺,R
b2,≻,I,R


## Optimistic and pessimistic classification

Optimistic and pessimistic classification procedures are used to assign
alternatives to ordered categories separated by base profiles.
These procedures utilize outranking relations:
 - `>` : preferred
 - `<` : not preferred
 - `I` : indifferent
 - `R` : incomparable

between the alternatives and the base profiles.

- The **optimistic** procedure increases the level of base profiles till the lowest base profile which is "preferred" to alternative (b `>` a) is found. The alternative is assigned to the category having this base as upper bound. If no base is "preferred", the alternative is assigned to the category above the highest base profile.

- The **pessimistic** procedure decreases the level of base profiles till a base profile which is "not preferred" to alternative (b `<` a) is found. The alternative is assigned to the category having this base as lower bound. If no base is "not preferred", the alternative is assigned to the category below the lowest base profile

When the outranking relation between base and alternative is only of "preference" (b `>` a or b `<` a), the classification of the alternative gives the same result with optimistic and pessimistic procedure. For example, consider the outranking relation $b$ = ["<", "<", ">", ">", ">", ">"] for an alternative a. Both classifications (optimistic and pessimistic) classify alternative $a$ in $(b_1, b_2)$, i.e. $a ∈ (b_1, b_2)$.

When the outranking relation between alternative and base profile is "indifferent" (`I`) or "incomparable" (`R`), the pessimistic procedure assigns the alternative in a lower category than the optimistic procedure. For example, consider the outranking relation $b$ = ["<", "<", "I", "R", ">", ">"] for an alternative $a$:

- Optimistic classification starts from lowest base profile $b_0$ = "<" and increases the index of $b$ till $b_4$ = ">". The alternative is classified in $(b_3, b_4)$, i.e. $a ∈ (b_3, b_4).$

- Pessimistic classification starts from highest base profile $b_{-1}$ = ">" and decreases the index of $b$ till $b_1$ = "<". The alternative is classified in $(b_1, b_2)$, i.e. $a ∈ (b_1, b_2).$

In [18]:
optimistic, pessimistic = et.classify(outranking)

In [19]:
print("\nOptimistic classification")
optimistic


Optimistic classification


,a1,a2,a3
b1 ≻,1.0,NaN,NaN
"(b1, b2)",NaN,NaN,NaN
b2 ≺,NaN,1.0,1.0


In [20]:
print("\nPessimistic classification")
pessimistic


Pessimistic classification


,a1,a2,a3
b1 ≻,1.0,NaN,1.0
"(b1, b2)",NaN,1.0,NaN
b2 ≺,NaN,NaN,NaN


## Raking and sorting

The optimistic and pessimistic classifications can be:
- ranked, by indicating for each alternative the class to which it belongs;

- sorted, by indicating for each class the alternatives that belong to it.

In [21]:
pessi_rank = et.rank(pessimistic)
print('\nPessimistic ranking')
pessi_rank.to_frame()


Pessimistic ranking


,0
a1,∈ b1 ≻
a2,"∈ (b1, b2)"
a3,∈ b1 ≻


In [22]:
pessi_sort = et.sort(pessimistic)
print('\nPessimistic sorting')
pessi_sort.to_frame()


Pessimistic sorting


,0
b1 ≻,"[a1, a3]"
"(b1, b2)",[a2]
b2 ≺,[]
